In [1]:
!nvidia-smi -L 

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-95b2c041-91ce-e912-792b-f083c5323ec8)


In [2]:
# connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers
!pip install datasets

In [40]:
%cd '/content/gdrive/MyDrive/BERT_copy'
!ls

/content/gdrive/MyDrive/BERT_copy
BERT_architecture.py  evaluate-v2.0.py	     runs	      train-v1.1.json
BERT_viola.ipynb      __pycache__	     run_tf_squad.py  utils_qa.py
dev-v1.1.json	      run_qa_beam_search.py  squad_v2_local
evaluate-v1.1.py      run_qa.py		     trainer_qa.py


In [ ]:
# # run squad1.1
# !python run_qa.py \
#   --model_name_or_path distilbert-base-uncased \
#   --dataset_name squad \
#   --do_train \
#   --do_eval \
#   --per_device_train_batch_size 12 \
#   --learning_rate 3e-5 \
#   --num_train_epochs 1 \
#   --max_seq_length 384 \
#   --doc_stride 128 \
#   --overwrite_output_dir \
#   --output_dir /tmp/debug_squad/

In [ ]:
# run squad2.0
!python run_qa.py \
  --model_name_or_path distilbert-base-uncased \
  --dataset_name squad_v2 \
  --version_2_with_negative \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 24 \
  --learning_rate 3e-5 \
  --num_train_epochs 2 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --overwrite_output_dir \
  --output_dir /tmp/debug_squad/ \
  --save_steps 2000

2020-12-09 02:30:02.711042: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/09/2020 02:30:04 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/09/2020 02:30:04 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/tmp/debug_squad/', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=24, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec09_02-30-04_badd818bdb9e', logging_first_step=False, logging_steps=50

In [27]:
import math
import os
import warnings
from dataclasses import dataclass
from typing import Optional, Tuple

import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

from transformers import BertTokenizer, BertModel, BertPreTrainedModel
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer, DistilBertPreTrainedModel

from transformers.modeling_outputs import (
    BaseModelOutputWithCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    MaskedLMOutput,
    MultipleChoiceModelOutput,
    NextSentencePredictorOutput,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutput,
    TokenClassifierOutput,
)
class pBLSTM(nn.Module):
    '''
    Pyramidal BiLSTM
    The length of utterance (speech input) can be hundereds to thousands of frames long.
    The Paper reports that a direct LSTM implementation as Encoder resulted in slow convergence,
    and inferior results even after extensive training.
    The major reason is inability of AttendAndSpell operation to extract relevant information
    from a large number of input steps.
    '''
    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bidirectional=True, batch_first=True)

    def forward(self, seq_unpacked):
        '''
        :param x :(N, T) input to the pBLSTM
        :return output: (N, T, H) encoded sequence from pyramidal Bi-LSTM 
        '''

        # seq_unpacked, lens_unpacked = utils.rnn.pad_packed_sequence(x, batch_first=True)

        if seq_unpacked.shape[1] % 2 == 1:
            seq_unpacked = seq_unpacked[:, :-1, :]

        x = seq_unpacked.reshape(seq_unpacked.shape[0], seq_unpacked.shape[1]//2, seq_unpacked.shape[2]*2)

        outputs, _ = self.blstm(x)

        return outputs

In [ ]:
# attempt 1 of writing our model
import math
import os
import warnings
from dataclasses import dataclass
from typing import Optional, Tuple

import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

from transformers import BertTokenizer, BertModel, BertPreTrainedModel

from transformers.modeling_outputs import (
    BaseModelOutputWithCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    MaskedLMOutput,
    MultipleChoiceModelOutput,
    NextSentencePredictorOutput,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutput,
    TokenClassifierOutput,
)


class BERT_CNN(BertPreTrainedModel):
    '''
    ATTEMPT NO. 1
    '''
    def __init__(self, config):
        # super(BERT_CNN, self).__init__()
        super().__init__(config)
        self.num_labels = config.num_labels

        # layer1: BERT
        # pretrained_weights='bert-base-uncased'
        # self.tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
        self.bert = BertModel(config, add_pooling_layer=False)

        # layer2: LSTM - according to the Stanford paper: https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/default/15848021.pdf
        hidden_dim = 256 # user defined
        input_dim = config.hidden_size # warning! might need to change, 768 = sequence_output.shape[2]
        self.lstm = nn.LSTM(input_size=config.hidden_size, hidden_size=hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        
        # layer3: CNN
        # WARNING! CNN dim's not yet working 
        # self.cnn = nn.Conv2d(in_channels=1, out_channels=hidden_cnn, kernel_size=3)

        # layer4: Linear Output Layer
        # use this if you want the cnn layer
        # self.qa_outputs = nn.Linear(hidden_cnn, vocab_size)
        # use this if omit the cnn layer
        self.qa_outputs = nn.Linear(2*hidden_dim, config.num_labels)
        self.init_weights()

    # WARNING! not sure if it would work for text in a batch 
    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,):
      
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]
        # 1: encode text and send to bert layer
        # bert_output shape: (batch_size, sequence_length, hidden_size)
        # encoded_input = self.tokenizer(text, return_tensors='pt')
        # output = self.model(**encoded_input)
        # bert_output = output[0]

        # 2: lstm layer
        # input of shape (batch, seq_len, input_size)
        # output from lstm has size (batch, seq_len, 2*input_size)
        lstm_output, _ = self.lstm(sequence_output)

        # 3: cnn layer
        # cnn input (N, Cin, Hin, Win)
        # cnn input (N, Cout, Hout, Wout)
        # not sure abou the dimensions yet
        # cnn_input = lstm_output.unsqueeze(1) # did this according to the Stanford paper, need more investigation
        # cnn_output = cnn(cnn_input)

        # final linear layer
        # WARNING! I assume we output text using argmax (greedy search)
        # maybe we only need the logits to calculate F1 score
        logits = self.qa_outputs(lstm_output)
        # vocab_index = logits.argmax(dim=2)

        # found this piece of code from BERT database: https://huggingface.co/transformers/_modules/transformers/modeling_bert.html#BertForTokenClassification
        # not sure why they added those
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        total_loss = None
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions.clamp_(0, ignored_index)
            end_positions.clamp_(0, ignored_index)

            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2

        if not return_dict:
            output = (start_logits, end_logits) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

        # ultimately we want to return a loss


In [ ]:
self.distilbert = DistilBertModel(config)
self.blstm = nn.LSTM(input_size=config.dim, hidden_size=256, num_layers=1, bidirectional=True, batch_first=True)
self.dropout = nn.Dropout(config.qa_dropout)
self.pblstm = pBLSTM(input_dim=256*4, hidden_dim=256)
self.qa_outputs = nn.Linear(256*4, config.num_labels)

assert config.num_labels == 2
self.init_weights()

In [39]:


pretrained_weights='bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

blstm = nn.LSTM(input_size=768, hidden_size=256, num_layers=1, bidirectional=True, batch_first=True)
dropout = nn.Dropout(0.5)
pblstm = pBLSTM(input_dim=256*4, hidden_dim=256)
qa_outputs = nn.Linear(256*2, 3000)

text = "let us try to see what it outputs"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

hidden_states = output[0] # last hidden state

lstm_output, _ = blstm(hidden_states)
hidden_states = dropout(lstm_output)  # (bs, max_query_len, dim)
hidden_states = pblstm(hidden_states)
logits = qa_outputs(hidden_states)  # (bs, max_query_len, 2)
# # lstm layer
# # input of shape (batch, seq_len, input_size)
# # output from lstm has size torch.Size([1, 5, 512])
# hidden_dim = 256
# input_dim = sequence_output.shape[2]
# lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
# lstm_output, _ = lstm(sequence_output)

# # # cnn input (N, Cin, Hin, Win)
# # # cnn input (N, Cout, Hout, Wout)
# # # not sure yet
# # cnn_input = lstm_output.unsqueeze(1) 
# # hidden_cnn = 512
# # cnn = nn.Conv2d(in_channels=1, out_channels=hidden_cnn, kernel_size=3)
# # cnn_output = cnn(cnn_input)

# # final linear layer
# vocab_size = 30522
# qa_outputs = nn.Linear(2*hidden_dim, vocab_size)
# logits = qa_outputs(lstm_output)
# # get max? Assuming greedy search
# print(logits.argmax(dim=2))
# # start_logits, end_logits = logits.split(1, dim=-1)
# # start_logits = start_logits.squeeze(-1)
# # end_logits = end_logits.squeeze(-1)